## Fetch the data

In [1]:
#lime
!pip install lime -q
import lime
from lime.lime_text import LimeTextExplainer
import lime.lime_tabular

# for hydrating tweet ids
!pip install twarc -q

#utils
import os
import re
import json
import random
from datetime import datetime
from shutil import copyfile
import string
from string import ascii_letters
from collections import Counter, defaultdict

#math, data
import math
import numpy as np
from numpy import inf
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import  word_tokenize 
from textblob import TextBlob

#plots, image
import cv2
from skimage import io
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

#ml, nlp
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA, TruncatedSVD

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

     |████████████████████████████████| 276kB 8.8MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
from IPython.display import clear_output
!rm -rf RawData
!rm -rf Junk
!rm -rf Classified
!gdown https://drive.google.com/uc?id=122tFro5g5u08M_DwLwYn5wDfHOR8BT48
!unzip RawData.zip
!rm RawData.zip
clear_output()

In [3]:
df = pd.read_csv('RawData/covid19_infodemic_english_data.tsv', sep='\t')

In [4]:
len(df)

504

In [5]:
df_1 = df.loc[df['q1_label'] == 'yes']
df_1 = df.loc[(df['q2_label'] == '4_yes_probably_contains_false_info') | (df['q2_label'] == '5_yes_definitely_contains_false_info')]
df_1 = df.loc[(df['q5_label'] == 'yes_not_urgent') | (df['q5_label'] == 'yes_very_urgent')]
df_1.head()

,tweet_id,text,q1_label,q2_label,q3_label,q4_label,q5_label,q6_label,q7_label
4,1241447017945223169,This is the face of someone who just spent 9 h...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,no_not_interesting
11,1240676575168921600,"So, the last week I have been battling COVID-1...",yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,2_no_probably_not_harmful,yes_not_urgent,no_not_harmful,yes_blame_authorities
13,1240768501452697600,He told wealthy friends. Then cashed out a coo...,yes,3_not_sure,5_yes_definitely_of_interest,5_yes_definitely_harmful,yes_not_urgent,yes_rumor_conspiracy,yes_blame_authorities
15,1241337295606657033,Korea finished developing the 10 minute Covid-...,yes,2_no_probably_contains_no_false_info,5_yes_definitely_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,yes_discusses_action_taken
19,1241057919128526856,"THREAD: I mostly talk about movies on here, bu...",yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,yes_discusses_action_taken


In [6]:
# We only need the Tweet ID's
tweets_fake_1 = df_1['tweet_id']
print("This dataset contains",df_1.shape[0], "Tweet ID's")

This dataset contains 160 Tweet ID's


In [7]:
def read_content_from_folder(dir_name): 
  fake_claims = pd.read_csv(os.path.join(dir_name, 'ClaimFakeCOVID-19.csv'))['title']
  fake_claims = pd.DataFrame({'content':fake_claims})
  fake_claims_tweets = pd.read_csv(os.path.join(dir_name, 'ClaimFakeCOVID-19_tweets.csv'))['tweet_id']
  fake_news =  pd.read_csv(os.path.join(dir_name, 'NewsFakeCOVID-19.csv'))[['title', 'content']]
  fake_news['content'] = fake_news['title'].replace(np.nan, '', regex=True) + ' ' + fake_news['content'].replace(np.nan, '', regex=True)
  fake_news = pd.DataFrame({'content':fake_news['content']})
  fake_news_tweets = pd.read_csv(os.path.join(dir_name, 'NewsFakeCOVID-19_tweets.csv'))['tweet_id']

  real_claims = pd.read_csv(os.path.join(dir_name, 'ClaimRealCOVID-19.csv'))['title']
  real_claims = pd.DataFrame({'content':real_claims})
  real_claims_tweets = pd.read_csv(os.path.join(dir_name, 'ClaimRealCOVID-19_tweets.csv'))['tweet_id']
  real_news =  pd.read_csv(os.path.join(dir_name, 'NewsRealCOVID-19.csv'))[['title', 'content']]
  real_news['content'] = real_news['title'].replace(np.nan, '', regex=True) + ' ' + real_news['content'].replace(np.nan, '', regex=True)
  real_news = pd.DataFrame({'content':real_news['content']})
  real_news_tweets = pd.read_csv(os.path.join(dir_name, 'NewsRealCOVID-19_tweets.csv'))['tweet_id']

  return [fake_claims, fake_claims_tweets, fake_news, fake_news_tweets, real_claims, real_claims_tweets, real_news, real_news_tweets]

l1 = read_content_from_folder('RawData/05-01-2020/')
l2 = read_content_from_folder('RawData/07-01-2020/')

all_dfs = []

# merging two lists, reset index
for i, dataframe in enumerate(l1):
  all_dfs.append(dataframe.append(l2[i], ignore_index=True))

fc, fct, fn, fnt, rc, rct, rn, rnt = all_dfs

In [8]:
fct = tweets_fake_1.append(fct, ignore_index=True)

In [9]:
# code from: https://github.com/susanli2016/NLP-with-Python/blob/master/Fake_News_LogReg.ipynb
df = pd.read_csv('RawData/corona_fake.csv')

df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)

df.loc[5,'label'] = 'FAKE'
df.loc[15,'label'] = 'TRUE'
df.loc[43,'label'] = 'FAKE'
df.loc[131,'label'] = 'TRUE'
df.loc[242,'label'] = 'FAKE'

df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)

df['title_text'] = df['title'] + ' ' + df['text']
def preprocessor(text):
    
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = text.lower()
    return text
    
df['title_text'] = df['title_text'].apply(preprocessor)
df['label'].value_counts()

TRUE    586
FAKE    578
Name: label, dtype: int64

In [10]:
df_t = pd.DataFrame({'content':df.loc[df['label']=='TRUE','title_text']})
df_f = pd.DataFrame({'content':df.loc[df['label']=='FAKE','title_text']})

In [11]:
fn = fn.append(df_f, ignore_index=True)
rn = rn.append(df_t, ignore_index=True)

In [12]:
bot_user_id = pd.read_csv("RawData/cresci_fake_users.txt", header=None)
real_user_id = pd.read_csv("RawData/cresci_real_users.txt", header=None)

In [13]:
dataframes = [("fake_claims", fc), ("fake_claims_tweets", fct), ("fake_news", fn), ("fake_news_tweets", fnt), 
              ("real_claims", rc), ("real_claims_tweets", rct), ("real_news", rn), ("real_news_tweets", rnt),
              ("bot_users", bot_user_id), ("real_users", real_user_id)]

[print(i[0], len(i[1])) for i in dataframes]
print()

fake_claims 28
fake_claims_tweets 644
fake_news 1416
fake_news_tweets 10416
real_claims 338
real_claims_tweets 7795
real_news 3303
real_news_tweets 141548
bot_users 3455
real_users 3474



In [14]:
# you can ignore this variable
# this will always be False if you are not the author of this notebook
author = True

# for temps files
!mkdir Junk

if not author:
  tweets_fake = fnt.append(fct, ignore_index=True)
  tweets_true = rnt.append(rct, ignore_index=True)

  # we combine all found tweet IDS
  print(len(tweets_fake), len(tweets_true))

  with open("Junk/tweets_fake.txt", "w") as f:
      for line in tweets_fake:
          f.write(str(line)+"\n") 

  with open("Junk/tweets_real.txt", "w") as f:
      for line in tweets_true:
          f.write(str(line)+"\n") 

  copyfile("RawData/cresci_fake_users.txt", "Junk/cresci_fake_users.txt")
  copyfile("RawData/cresci_real_users.txt", "Junk/cresci_real_users.txt")

  # Need a Twitter dev account and keys
  !twarc configure

  !mkdir Classified

  !twarc hydrate Junk/tweets_fake.txt > Classified/tweets_fake.jsonl
  !twarc hydrate Junk/tweets_real.txt > Classified/tweets_real.jsonl
  !twarc users Junk/cresci_real_users.txt > Classified/real_users.jsonl
  !twarc users Junk/cresci_fake_users.txt > Classified/fake_users.jsonl
else:
  # only the author can retrieve the prepopulated data from a personal Google drive folder
  !gdown https://drive.google.com/uc?id=1eSc4OM4-AOh0E829ShObkHk_NpiGGlL-
  !unzip Classified.zip
  !rm Classified.zip

def read_json(filename):
  tweets = []
  with open(filename) as f:
    for line in f:
      tweets.append(json.loads(line))
  return tweets

Downloading...
From: https://drive.google.com/uc?id=1eSc4OM4-AOh0E829ShObkHk_NpiGGlL-
To: /content/Classified.zip
68.1MB [00:01, 35.4MB/s]
Archive:  Classified.zip
   creating: Classified/
  inflating: Classified/tweets_real.jsonl  
  inflating: Classified/fake_users.jsonl  
  inflating: Classified/real_users.jsonl  
  inflating: Classified/tweets_fake.jsonl  


In [15]:
# these contain both Tweet data and user data
tweets_fake_ = read_json('Classified/tweets_fake.jsonl')
tweets_real_ = read_json('Classified/tweets_real.jsonl')

# these only contain user data
users_fake_ = read_json('Classified/fake_users.jsonl')
users_real_ = read_json('Classified/real_users.jsonl')

#sanity check
print(len(tweets_fake_))
print(len(tweets_real_))
print(len(users_fake_))
print(len(users_real_))

10165
142761
2804
2892


In [16]:

tweet_columns = ["id", "source", "full_text", "retweet_count", "favorite_count", "possibly_sensitive", "entities", "created_at"]
user_columns = ["id", "name", "screen_name", "has_extended_profile", "location", "description", "url", "protected", "followers_count", "friends_count", "listed_count", "created_at", "favourites_count", "verified", "statuses_count", "profile_image_url_https", "default_profile", "default_profile_image"]

# this function will be used to convert the json files that contain
# both the tweet level and user level data to a Pandas dataframe
def json_to_dataframe(json, amount):
  tweets = []
  users = []
  for i, t in enumerate(json):
    tweet_row = []
    user_row = []
    for _, tc in enumerate(tweet_columns):
      if (tc == "possibly_sensitive"):
        if "possibly_sensitive" in t:
          tweet_row.append(t["possibly_sensitive"])
        else:
          tweet_row.append(False)
      else:
        tweet_row.append(t[tc])    
    for _, tu in enumerate(user_columns):
      user_row.append(t['user'][tu])
    tweets.append(tweet_row)
    users.append(user_row)
  df_t = pd.DataFrame(data= tweets, columns=tweet_columns)
  df_u = pd.DataFrame(data=users, columns=user_columns)

  return df_t[0:amount], df_u[0:amount]

# this function will be used to convert only the json files
# that contain user level data to a Pandas dataframe
def user_json_to_dataframe(json):
  users = []
  for i, u in enumerate(json):
    user_row = []
    for _, tu in enumerate(user_columns):
      user_row.append(u[tu])
    users.append(user_row)
  df_u = pd.DataFrame(data=users, columns=user_columns)
  return df_u

In [17]:
#https://towardsdatascience.com/automatically-detect-covid-19-misinformation-f7ceca1dc1c7

#https://medium.com/pew-research-center-decoded/detecting-subjectivity-and-tone-with-automated-text-analysis-tools-5f0e662224b8

def calculate_is_subjective(mined_data):
  def textblob_adj(text):
      blobed = TextBlob(text)
      counts = Counter(tag for word,tag in blobed.tags)
      adj_list = []
      adv_list = []
      adj_tag_list = ['JJ','JJR','JJS']
      adv_tag_list = ['RB','RBR','RBS']
      for (a, b) in blobed.tags:
        if b in adj_tag_list:
          adj_list.append(a)
        elif b in adv_tag_list:
          adv_list.append(a)
        else:
          pass
      return adj_list, adv_list, counts['JJ']+counts['JJR']+counts['JJS'], counts['RB']+counts['RBR']+counts['RBS'], counts['NNP']
    
  mined_data['subjective'] = 0
  mined_data['n_nnp'] = 0

  for i, k in mined_data.iterrows():
    adj, adv, adj_c, adv_c, nnp_c = textblob_adj(k['full_text'])
    mined_data['n_nnp'].loc[i] = nnp_c
    if (adj_c >= 2 and adv_c >=1):
      mined_data['subjective'].loc[i] = 1
  return mined_data

def preprocess_tweet(mined_data, keep_all=True):
  def add_amount_features(col_name, char):
    mined_data[col_name] = 0
    for i, s in enumerate(mined_data['full_text']):
      n_q = 0
      for k in s:
        if (k == char):
          n_q +=1
      mined_data[col_name].loc[i] = n_q

  add_amount_features('n_question', '?')
  add_amount_features('n_exclaim', '!')
  add_amount_features('n_hash', '#')
  mined_data['n_uppercase'] = mined_data['full_text'].str.findall(r'[A-Z]').str.len()
  mined_data['length'] = mined_data['full_text'].str.len()
  mined_data['created_at_tweet'] = mined_data['created_at']
    
  mined_data['has_link'] = False
  for i, s in enumerate(mined_data['full_text']):
    mined_data.loc[i, 'has_link'] = ('t.co' in s) or ('http' in s)
  mined_data['has_link'] = mined_data['has_link'].astype(int)
  mined_data['possibly_sensitive'] = mined_data['possibly_sensitive'].astype(int)
  mined_data = calculate_is_subjective(mined_data)

  return mined_data

def preprocess_users(mined_data, keep_all=False):
  
  mined_data['bio_length'] = mined_data['description'].apply(len)
  mined_data['username_length'] = mined_data['screen_name'].apply(len)
  mined_data['name_length'] = mined_data['name'].apply(len)
  date_format = "%a %b %d %H:%M:%S +%f %Y" 
  mined_data['age_account_days'] = 0
  for i, r in enumerate(mined_data['created_at']):
    a = datetime.now()
    b = datetime.strptime(r, date_format)
    mined_data['age_account_days'].iloc[i] = (a - b).days

  mined_data['created_at_user'] = mined_data['created_at']
  
  mined_data['friends_count'] = mined_data['friends_count'].replace([0],1)
  mined_data['followers_friends_ratio'] = (mined_data['followers_count'] / mined_data['friends_count'])
  mined_data['age_account_days'] = mined_data['age_account_days'].replace([0],1)
  mined_data['likes_age_ratio'] = (mined_data['favourites_count'] / mined_data['age_account_days'])

  mined_data['verified'] = mined_data['verified'].astype(int)
  mined_data['has_location'] = mined_data['location'] != ''
  mined_data['has_location'] = mined_data['has_location'].astype(int)
  mined_data['default_profile_image'] = mined_data['default_profile_image'].astype(int)
  mined_data['default_profile'] = mined_data['default_profile'].astype(int)
  mined_data['protected'] = mined_data['protected'].astype(int)
  mined_data['has_url'] = mined_data['url'].astype(str) != "None"
  mined_data['has_url'] = mined_data['has_url'].astype(int)
 
  return mined_data

In [18]:
# these contain fake/real news and claims of Twitter users 
# as well as the users that posted them
amount = 10000
tweets_fake, tweets_fake_users = json_to_dataframe(tweets_fake_, amount)
tweets_real, tweets_real_users = json_to_dataframe(tweets_real_, amount)
tweets_fake = preprocess_tweet(tweets_fake)
tweets_real = preprocess_tweet(tweets_real)

# we preprocess the previous users and mine relevant features
tweets_fake_users = preprocess_users(tweets_fake_users)
tweets_real_users = preprocess_users(tweets_real_users)

# these will be used to train the social bot detector 
bots = preprocess_users(user_json_to_dataframe(users_fake_))
non_bots = preprocess_users(user_json_to_dataframe(users_real_))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [19]:

fake_news_articles = fn
real_news_articles = rn
print(len(fake_news_articles), "Fake and", len(real_news_articles), "real news articles (text)")

fake_claims = fc
real_claims = rc
print(len(fake_claims), "fake and", len(real_claims), "real website claims (text)")

print(len(tweets_fake), "fake and", len(tweets_real), "real news and claims (tweets)")

print("The corresponding", len(tweets_fake_users)+len(tweets_real_users) ,"users that posted these fake real claims and news (users)")

#print("The corresponding", len(tweets_fake_users_study)+len(tweets_real_users_study) ,"users that posted these fake real claims and news (users) - version for user study")
 
print(len(bots), "bots and", len(non_bots), "real users (users)")

1416 Fake and 3303 real news articles (text)
28 fake and 338 real website claims (text)
10000 fake and 10000 real news and claims (tweets)
The corresponding 20000 users that posted these fake real claims and news (users)
2804 bots and 2892 real users (users)


## Data mining

Add target attribute and concat all the DataFrames:

In [20]:
fake_news_articles["label"] = 1
real_news_articles["label"] = 0
fake_claims["label"] = 1
real_claims["label"] = 0
tweets_fake["label"] = 1
tweets_real["label"] = 0

tweets_fake = tweets_fake[["full_text", "label"]].copy()
tweets_real = tweets_real[["full_text", "label"]].copy()

dfs = [fake_news_articles,real_news_articles,fake_claims,real_claims,tweets_fake,tweets_real]

# Rename features
for df in dfs:
  df.columns = ["text", "target"]
  
# Concat dfs
dataset = pd.concat(dfs)

In [21]:
dataset.drop_duplicates(inplace=True)

In [22]:
dataset.head()

,text,target
0,Facebook posts shared in at least three countr...,1
1,Wisconsin is Òclearly seeing a decline in COVI...,1
2,Facebook posts claim a child who is infected w...,1
3,IndiaÕs Ministry of Home Affairs banning citiz...,1
4,"42 Democratic senators, plus two Independents,...",1


### Text mining

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [ ]:
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [ ]:
dataset["text"] = dataset["text"].apply(remove_contractions)

In [ ]:
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [ ]:
!pip install simpletransformers
!pip install emoji
!pip install itertools
clear_output()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sklearn
import itertools
import emoji

# Debug: Metadata not found (AttributeError)
try:
  from simpletransformers.classification import ClassificationModel
except(AttributeError):
  !pip install simpletransformers
  from simpletransformers.classification import ClassificationModel


In [ ]:
dataset["text"] = dataset["text"].apply(clean_dataset)

In [ ]:
dataset.drop_duplicates(subset=["text"], inplace=True)

In [ ]:
dataset.dropna(inplace=True)

In [23]:
dataset.to_csv("covid_clean_dataset_09_02_21.csv")
!cp covid_clean_dataset_09_02_21.csv /content/drive/MyDrive/Colab_Notebooks/Zenodo_DS_Project/

## Clean ZENODO dataset

In [ ]:
import pandas as pd
zenodo_tweets = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Zenodo_DS_Project/hydrated_tweets.csv", usecols=["text"])

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [ ]:
# Remove contractions: 
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

zenodo_tweets["text"] = zenodo_tweets["text"].apply(remove_contractions)

In [ ]:
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [ ]:
!pip install simpletransformers
!pip install emoji
!pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sklearn
import itertools
import emoji

In [ ]:
zenodo_tweets["text"] = zenodo_tweets["text"].apply(clean_dataset)

In [ ]:
zenodo_tweets.drop_duplicates(subset=["text"], inplace=True)

In [ ]:
zenodo_tweets.dropna(inplace=True)

In [ ]:
zenodo_tweets.to_csv("zenodo_covid_dataset_1.csv")
!cp zenodo_covid_dataset_1.csv /content/drive/MyDrive/Colab_Notebooks/Zenodo_DS_Project/